# Modelling and evaluation

Explantion of what we are doing

### Importing modules

In [14]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.dummy import DummyClassifier

from imblearn.over_sampling import SMOTE 


### Importing data

In [15]:
df = pd.read_csv('PreProcessedData.csv', sep=",")

df.info()

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534147 entries, 0 to 534146
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         534147 non-null  int64  
 1   State              534147 non-null  int64  
 2   Zip                534147 non-null  int64  
 3   NAICS              534147 non-null  int64  
 4   ApprovalFY         534147 non-null  int64  
 5   Term               534147 non-null  int64  
 6   NoEmp              534147 non-null  int64  
 7   NewExist           534147 non-null  int64  
 8   CreateJob          534147 non-null  int64  
 9   RetainedJob        534147 non-null  int64  
 10  FranchiseCode      534147 non-null  int64  
 11  UrbanRural         534147 non-null  int64  
 12  LowDoc             534147 non-null  int64  
 13  DisbursementGross  534147 non-null  float64
 14  MIS_Status         534147 non-null  int64  
 15  GrAppv             534147 non-null  float64
 16  Ra

,Unnamed: 0,State,Zip,NAICS,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,LowDoc,DisbursementGross,MIS_Status,GrAppv,Ratio
0,12,27,27502,44,2006,162,2,2,0,0,1,1,0,253400.0,1,253400.0,0.75
1,24,2,72223,72,2006,126,7,1,0,0,0,1,0,137300.0,1,137300.0,0.85
2,28,4,93010,61,2006,83,18,2,5,23,0,1,0,438541.0,1,100000.0,0.50
3,30,9,33186,23,2006,84,4,1,0,4,0,1,0,51440.0,1,35000.0,0.50
4,32,18,70130,53,2006,60,3,1,0,0,0,1,0,50000.0,1,50000.0,0.85


### Establish baseline

In [16]:
X = df[['State','Zip','NAICS','Term','NoEmp','NewExist','CreateJob','RetainedJob','FranchiseCode','UrbanRural','LowDoc','DisbursementGross','GrAppv','Ratio']]

y = df['MIS_Status']

X.columns

Index(['State', 'Zip', 'NAICS', 'Term', 'NoEmp', 'NewExist', 'CreateJob',
       'RetainedJob', 'FranchiseCode', 'UrbanRural', 'LowDoc',
       'DisbursementGross', 'GrAppv', 'Ratio'],
      dtype='object')

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

print(X_train.shape, y_train.shape,X_test.shape, y_test.shape)

(400610, 14) (400610,) (133537, 14) (133537,)


In [18]:
sum(y)/len(y)*100

75.59005292550552

In [19]:
#EVT. CUT DENNE DEL UD HVIS DET IKKE GIVER MENING
dummy = DummyClassifier()
dummy.fit(X_train, y_train)
dummy_y_pred = dummy.predict(X_test)

print("Accuracy on training set: {:.3f}".format(dummy.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(dummy.score(X_test, y_test)))

Accuracy on training set: 0.756
Accuracy on test set: 0.756


In [20]:
#EVT. CONFUSION MATRIX PÅ BAGGRUND AF DUMMY
#cm = confusion_matrix(y_test, dum_y_pred)
#print(cm)

In [21]:
sm = SMOTE(random_state=42)
X_SMOTE, y_SMOTE = sm.fit_resample(X, y)

sum(y_SMOTE)/len(y_SMOTE)

0.5

In [22]:
#New test and train set after SMOTE
X_train, X_test, y_train, y_test = train_test_split(X_SMOTE, y_SMOTE, random_state=42)

X_train

,State,Zip,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,LowDoc,DisbursementGross,GrAppv,Ratio
53452,10,30340,72,200,4,1,0,4,0,1,0,860000.000000,980000.0,0.75
74560,34,11363,44,50,4,1,0,4,0,1,0,180000.000000,100000.0,0.50
115844,37,97205,54,84,1,1,0,1,0,1,0,20000.000000,20000.0,0.50
684654,22,48167,43,61,3,1,0,3,0,1,0,55243.373840,60000.0,0.50
553256,3,85315,61,34,3,1,0,0,0,1,0,16961.367151,10000.0,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,34,13901,32,63,1,1,0,0,0,1,1,35000.000000,35000.0,0.80
365838,47,98270,72,240,25,1,0,25,0,1,0,238000.000000,238000.0,1.00
131932,15,46143,23,32,1,2,0,1,0,1,0,46796.000000,20000.0,0.50
671155,9,33171,45,57,5,1,5,9,0,1,0,45000.000000,45000.0,0.50


In [23]:
#Try dummy test again after SMOTE
dummy = DummyClassifier()
dummy.fit(X_train, y_train)
dummy_y_pred = dummy.predict(X_test)

print("Accuracy on training set: {:.3f}".format(dummy.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(dummy.score(X_test, y_test)))

Accuracy on training set: 0.500
Accuracy on test set: 0.499


# CLASSIFIERS (MODELS)

Explanation of process and procedure

### Logistic Regression